In [ ]:
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential  
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient  

from azure.search.documents.models import (
    VectorizableTextQuery, 
)

import os
from dotenv import load_dotenv  
load_dotenv(override=True)

AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')

client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,  
    api_version="2023-10-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

In [ ]:
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")  
credential = AzureKeyCredential(key)
endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")

def vector_text_search_indx(user_query:str,index_name="bytewax-index", top_results = 3):  
    query_vec = VectorizableTextQuery(text=user_query, k=3, fields="vector", exhaustive=True)  
    search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)
    search_results = search_client.search(      
        search_text=None,      
        vector_queries= [query_vec],    
        select=["id", "meta", "content"],    
        top=top_results    
    )    
    output_str = ''    
    for result in search_results:      
        # print(f"Parent ID: {result['parent_id']}")      
        # print(f"Chunk: {result['chunk']}")      
        # print(f"Score: {result['@search.score']}")      
        # print(f"Title: {result['title']}")   
        # output_str += f"Parent ID: {result['parent_id']}\nChunk: {result['chunk']}\nScore: {result['@search.score']}\nTitle: {result['title']}\n\n"  
        output_str=f"{result['content']}"  
    return output_str
  
def get_ans(user_query):
    # a=vector_text_search_indx(user_query,index_name='bytewax-index', top_results = 3)
    messages=[  
    {"role": "assistant", "content": "You are an AI assistant that helps in giving answers based on the provided data."},  
    {"role": "user", "content": 'Answer the query: {user_query} based on the below context:'+'/n'+vector_text_search_indx(user_query,index_name='bytewax-index', top_results = 3)}  
    ]
    model = os.getenv("AZURE_OPENAI_MODEL")
    response = client.chat.completions.create(
        model=model,
        messages = messages,
        temperature=0,
        max_tokens=1000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
        )
    ans=response.choices[0].message.content
    # Print the AI-generated response
    # print(f"{query_results}")
    return ans

get_ans("Provide a summary of the information in your database")